In [1]:
from pathlib import Path
from typing import cast, Any
from datetime import datetime
from dataclasses import dataclass
import re

import numpy as np
import numpy.typing as npt
from pisets import load_sound # type: ignore
import gigaam
from gigaam.model import GigaAMASR
from tqdm.auto import tqdm
import scipy

from asr_eval.models.gigaam import transcribe_with_gigaam_ctc, FREQ, encode
from asr_eval.ctc.forced_alignment import forced_alignment

In [2]:
model = cast(GigaAMASR, gigaam.load_model('ctc', device='cuda'))

In [3]:
video_path = Path('tmp/dion/kondrashuk_2025-07-09T12_01_04+03_00.mp4')
text_path = video_path.with_suffix('.txt')

waveform = cast(npt.NDArray[np.floating[Any]], load_sound(video_path))

In [4]:
SEGMENT_SIZE_SEC = 20
SEGMENT_SHIFT_SEC = 6
TICK_SIZE = 1 / FREQ

total_len_sec = len(waveform) / 16_000
total_ticks = int(total_len_sec / TICK_SIZE)
print(f'{total_len_sec = }, {total_ticks = }')

SAMPLING_RATE = gigaam.preprocess.SAMPLE_RATE

class LogProbsWindow:
    def __init__(
        self,
        start_time: float,  # can be negative
        end_time: float,
        waveform_16k: npt.NDArray[np.floating[Any]],
        model: GigaAMASR
    ):
        total_len_sec = len(waveform_16k) / SAMPLING_RATE
        
        clipped_start_time = np.clip(start_time, 0, total_len_sec)
        clipped_end_time = np.clip(end_time, 0, total_len_sec)
        
        self.clipped_start_ticks = int(clipped_start_time * FREQ)
        clipped_end_ticks = int(clipped_end_time * FREQ)
        
        clipped_start_pos = int(self.clipped_start_ticks * TICK_SIZE * SAMPLING_RATE)
        clipped_end_pos = int(clipped_end_ticks * TICK_SIZE * SAMPLING_RATE)
        
        waveform_chunk = waveform[clipped_start_pos : clipped_end_pos]
        self.log_probs = transcribe_with_gigaam_ctc(model, [waveform_chunk])[0].log_probs
        
        assert np.allclose(self.clipped_start_ticks + len(self.log_probs), clipped_end_ticks, atol=1.1)
        self.clipped_end_ticks = self.clipped_start_ticks + len(self.log_probs)
        
        clip_ratio_start = (clipped_start_time - start_time) / (end_time - start_time)
        clip_ratio_end = (clipped_end_time - start_time) / (end_time - start_time)
        
        self.weights = scipy.stats.beta.pdf(
            np.linspace(clip_ratio_start, clip_ratio_end, num=len(self.log_probs)), a=5, b=5
        )
        self.weights /= self.weights.max()

def average_logp_windows(windows: list[LogProbsWindow]) -> npt.NDArray[np.floating[Any]]:
    max_ticks = max(window.clipped_end_ticks for window in windows)
    
    sum_weights = np.zeros(max_ticks)
    for window in windows:
        sum_weights[window.clipped_start_ticks:window.clipped_end_ticks] += window.weights

    averaged_log_probs = np.zeros((max_ticks, windows[0].log_probs.shape[1]))
    for window in windows:
        span = slice(window.clipped_start_ticks, window.clipped_end_ticks)
        averaged_log_probs[span] += (
            window.log_probs * (window.weights / sum_weights[span])[:, None]
        )
    
    return averaged_log_probs

windows: list[LogProbsWindow] = []

for center in tqdm(np.arange(0, total_len_sec, step=SEGMENT_SHIFT_SEC)):
    start = center - SEGMENT_SIZE_SEC / 2  # can be negative
    end = center + SEGMENT_SIZE_SEC / 2
    windows.append(LogProbsWindow(start, end, waveform, model))

averaged_log_probs = average_logp_windows(windows)
averaged_log_probs.shape

total_len_sec = 852.096, total_ticks = 21302


  0%|          | 0/143 [00:00<?, ?it/s]

(21303, 34)

In [5]:
def filter_encodable_tokens(text: str, model: GigaAMASR) -> str:
    assert model.decoding.tokenizer.charwise
    text = text.lower().replace('ё', 'е').replace('-', ' ')
    return ''.join(char for char in text if char in model.decoding.tokenizer.vocab)

@dataclass
class DionSegment:
    approx_start_time: float
    speaker: str
    text: str
    text_start_pos: int
    text_end_pos: int

joined_text = ''
segments: list[DionSegment] = []
for record in text_path.read_text(encoding='utf-8-sig').split('\n\n'):
    start_time_str, speaker, _, text = record.split('\n')
    
    text = text[2:].strip()  # skip "- " prefix in the file
    text = filter_encodable_tokens(text, model)
    
    try:
        start_time = datetime.strptime(start_time_str, '%H:%M:%S') - datetime(1900, 1, 1)
    except ValueError:
        start_time = datetime.strptime(start_time_str, '%M:%S') - datetime(1900, 1, 1)
    
    if len(joined_text):
        joined_text += ' '
    joined_text += text
    
    segments.append(DionSegment(
        approx_start_time=start_time.total_seconds(),
        speaker=speaker,
        text=text,
        text_start_pos=len(joined_text) - len(text),
        text_end_pos=len(joined_text),
    ))
    
display(segments[:7])
print(joined_text[:1000])

[DionSegment(approx_start_time=11.0, speaker='Анастасия', text='георгий', text_start_pos=0, text_end_pos=7),
 DionSegment(approx_start_time=11.0, speaker='Анастасия', text='тебя не слышного', text_start_pos=8, text_end_pos=24),
 DionSegment(approx_start_time=21.0, speaker='Анастасия', text='так', text_start_pos=25, text_end_pos=28),
 DionSegment(approx_start_time=25.0, speaker='Анастасия', text='вадим', text_start_pos=29, text_end_pos=34),
 DionSegment(approx_start_time=26.0, speaker='Анастасия', text='у тебя есть какие то вопросы', text_start_pos=35, text_end_pos=63),
 DionSegment(approx_start_time=28.0, speaker='Георгий', text='агайнело', text_start_pos=64, text_end_pos=72),
 DionSegment(approx_start_time=30.0, speaker='Георгий', text='всем привет', text_start_pos=73, text_end_pos=84)]

георгий тебя не слышного так вадим у тебя есть какие то вопросы агайнело всем привет да привет так что давайте начинать наверное или мы под ждем еще да я и спросила может быть есть какие то вопросы как с которых имеет смысл начать да но у меня основное наверное хотелось бы там поговорить про реот модельку вот если я там еще ничего не забыл вот угу ну и по остальным там статусам пробежаться по быстрому а так сегодня кажется можем быстро разойтись ну да давай сначала по обзорам собственно завтра по длинному контексту потом у нас на следующую неделю предварительно с шестнадцатого по восемнадцатое тоже от вас нужно будет время потому что там будет ну напомню полное название твою темы по мойте агентам системе а да вот то есть с шестнадцатого по восемнадцатое посмотрите и потом будем готовы в период тридцатого июля по первое августа провести последний большой третий темь да хорошо окей ну там я посмотрю тогда по слоту мы отправили также какой то раз угу угу да хорошо вот ну то есть собственн

In [6]:
idx_per_frame, scores_per_frame, true_tokens_pos = forced_alignment(
    log_probs=averaged_log_probs,
    true_tokens=encode(model, joined_text),
    blank_id=model.decoding.blank_id,
)

In [7]:
@dataclass
class TimedWord:
    start_time: float
    end_time: float
    speaker: str
    text: str
    uncertainty_score: float

timed_words: list[TimedWord] = []
for segment in segments:
    
    start_frame, _ = true_tokens_pos[segment.text_start_pos]
    uncertainty_score = abs(start_frame / FREQ - segment.approx_start_time)
    assert uncertainty_score < 5, (
        'Forced alignment failed to match Dion timings'
    )
    
    for match in re.finditer(r'\w+', segment.text):
        start_frame, _ = true_tokens_pos[segment.text_start_pos + match.start()]
        _, end_frame = true_tokens_pos[segment.text_start_pos + match.end() - 1]
        timed_words.append(TimedWord(
            start_time=start_frame / FREQ,
            end_time=end_frame / FREQ,
            speaker=segment.speaker,
            text=match.group(),
            uncertainty_score=round(uncertainty_score, 3),
        ))
        
timed_words[:7]

[TimedWord(start_time=9.08, end_time=9.56, speaker='Анастасия', text='георгий', uncertainty_score=1.92),
 TimedWord(start_time=9.8, end_time=9.96, speaker='Анастасия', text='тебя', uncertainty_score=1.2),
 TimedWord(start_time=10.12, end_time=10.2, speaker='Анастасия', text='не', uncertainty_score=1.2),
 TimedWord(start_time=10.28, end_time=10.76, speaker='Анастасия', text='слышного', uncertainty_score=1.2),
 TimedWord(start_time=19.08, end_time=19.32, speaker='Анастасия', text='так', uncertainty_score=1.92),
 TimedWord(start_time=23.48, end_time=23.8, speaker='Анастасия', text='вадим', uncertainty_score=1.52),
 TimedWord(start_time=23.92, end_time=23.96, speaker='Анастасия', text='у', uncertainty_score=2.08)]